Здесь мы звпускаем прогон модели на части тестовой выборке для демонстрации модели на новых данных

In [1]:
import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer, pipeline
from peft import AutoPeftModelForSeq2SeqLM # type: ignore
from tqdm import tqdm

tqdm.pandas()

In [2]:
df = pd.read_parquet('./__input__/test.parquet').drop('__index_level_0__', axis=1)
df = df.drop_duplicates().dropna()
df = df.sample(200)

df

,nmId,text,responder,type,product_name,product_category_2,product_category_1,product_color,product_description,product_brand
135773,1068588,Здравствуйте. Данная модель рассчитана на темп...,Магазин,Вопрос,Комбинезон,Комбинезоны для малышей,Одежда для малышей,синий,,Bell bimbo
101538,28885528,Добрый день! Большое спасибо за Ваш отзыв! Мы ...,Магазин,Отзыв,Юбка женская летняя для девочки,Юбки,Одежда,серый,"Юбка LUNA миди, с разрезом. Ткань - вискоза 10...",Luna style
371650,8976468,В отзывах 28.08. Модель представлена ЦВЕТ СИНИ...,Клиент,Вопрос,"Платье ""Школьница""",Платья,Одежда,"антрацитовый, белый, серый, черный",Платье для девочки выполнено из теплого хлопко...,Апрель
102265,3883790,"Качество вроде нормальное, ткань тонкая, легка...",Клиент,Отзыв,Юбка,Юбки,Одежда,"красный, черный",,RAMANTI
337147,2992008,Хотелось бы уточнить есть ли у этой блузки вор...,Клиент,Вопрос,Блузка,Блузки,Одежда,"темно-синий, синий, зеленый, темно-зеленый, кр...",,OLBE
...,...,...,...,...,...,...,...,...,...,...
14740,11977609,"Жакет приобрела вместе с юбкой, качество изгот...",Клиент,Отзыв,Жакет,Жакеты,Одежда,"темно-синий, белый",Способность жакета одновременно визуально прид...,Модный дом Виктории Тишиной
431080,13616454,"Невообразимо красивое кольцо! \nКамень ровный,...",Клиент,Отзыв,ювелирное кольцо,Ювелирные кольца,Ювелирные украшения,белый,Кольцо с натуральным лунным камнем! Вставка - ...,NARIMAN JEWELLERY
128626,4012784,Способ применения указан на стикере.,Магазин,Вопрос,Ампульная сыворотка с пептидами,Сыворотки,Красота,,Комплекс пептидов в составе сыворотки (олигопе...,La miso
173010,5592352,Добрый день! Да.,Магазин,Вопрос,"Интерьерная наклейка ""Бабочки"" с сильным клеев...",Наклейки интерьерные,Дом,белый,"Интерьерная наклейка ""Бабочки"" с сильным клеев...",Оранжевый Слоник


In [3]:
ds = Dataset.from_pandas(df)

In [4]:
def load_calssif(name: str):
    return pipeline(
        task='text-classification',
        model=ORTModelForSequenceClassification.from_pretrained(
            name,
            export=True,
            provider="CUDAExecutionProvider",
            use_io_binding=True
        ), # type: ignore
        tokenizer=AutoTokenizer.from_pretrained(
            name,
            useFast=True
        ),
        device='cuda:0'
    )

In [5]:
tox_class_pipe = load_calssif('cointegrated/rubert-tiny-toxicity')
emot_class_pipe = load_calssif('cointegrated/rubert-tiny2-cedr-emotion-detection')

2024-12-23 20:06:50.103659366 [W:onnxruntime:, transformer_memcpy.cc:74 ApplyImpl] 3 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.


In [6]:
def predict(row):
    return {
        **row,
        'toxicity': tox_class_pipe(row['text'][:tox_class_pipe.tokenizer.model_max_length])[0]['label'], # type: ignore
        'emotions': emot_class_pipe(row['text'][:emot_class_pipe.tokenizer.model_max_length])[0]['label'] # type: ignore
    }

In [7]:

ds = ds.map(predict)

Parameter 'function'=<function predict at 0x710a9abed8a0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [8]:
model = AutoPeftModelForSeq2SeqLM.from_pretrained(
    './models/25494ed9-f307-4293-a192-80678a7aca3a/model',
    device_map='cuda:0',
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    'ai-forever/ruT5-base',
    useFast=True
)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [9]:
def make_full_promt(e):
    return f'''
ответчик: {e['responder']}
тип: {e['type']}
название: {e['product_name']}
категория 2: {e['product_category_2']}
цвет: {e['product_color']}
бренд: {e['product_brand']}
описание: {e['product_description']}
токсичность: {e['toxicity']}
эиоциональность: {e['emotions']}
текст: {e['text']}
    '''


def generate_res(row):
    model_in = tokenizer(
        make_full_promt(row),
        truncation=True,
        max_length=1400,
        return_tensors='pt'
    ).to('cuda:0')

    model_out = model.generate(
        **model_in,
        max_new_tokens=300,
        top_k=1,
        do_sample=True,
    )[0]
    return {
        **row,
        'preds': tokenizer.decode(model_out, skip_special_tokens=True)
    }

In [10]:
ds = ds.map(generate_res)

ds

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Dataset({
    features: ['nmId', 'text', 'responder', 'type', 'product_name', 'product_category_2', 'product_category_1', 'product_color', 'product_description', 'product_brand', '__index_level_0__', 'toxicity', 'emotions', 'preds'],
    num_rows: 200
})

In [11]:
ds.to_parquet('./__output__/test_res.parquet')

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

382902

Вывод случайных 20 предложений в формате - {Изначальный текст} -> {После обработки моделью}

In [12]:
for d in ds.select(np.random.randint(0, len(ds), 20)):
    print('-------------------------')
    print()
    print(f'{d['text']} -> {d['preds']}')
    print()

-------------------------

Добрый день.Давно ищу эту пудру 13 оттенок HONEY BEIGE..Будет ли он в продаже? -> Здравствуйте! Будет ли в продаже эта пудра 13 оттенка?

-------------------------

Коврик бомба за свои деньги, при том что зайдя в магазин техники. Такой можно найти минимум от 1500.
Большое спасибо продавцу.
Очень надеюсь что до «умных» покупателей дойдет то что работная сторона черная и они перестанут портить рейтинг реально нормальным вещам) -> Коврик отличный, но за свои деньги он не стоит таких денег. За такую цену можно купить и за 1500. Но это не критично. Коврик не очень удобный, но если вы хотите купить, то лучше купить в подарок.

-------------------------

КАК БРАК НЕ ПОДТВЕРЖДЕН???? Мы сначала всем пунктом самовывоза во Владивостоке пытались  понять ЧТО ЭТО ТАКОЕ, а потом удивлялись - КАК ЭТО МОНО БЫЛО ПРИСЛАТЬ!!! 1. Крылья были припаяны в цепочками другой стороной, т.е. в месте соединения в центре колье были ДЛИННЫЕ КОНЧИКИ ПЕРЬЕВ - это НЕ БРАК????? 2. Страз в цент

Из этой демонстрации видно, что модель сохраняет информации из исходного текста, переписывая этот текст в более нейтральном ключе, но иногда может выдавать своеобразные речевые обороты и склонностью уходить в самоповтор